In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import torch
import pdb
from sklearn.metrics import roc_auc_score
# np.random.seed(2024)
# torch.manual_seed(2024)
import pdb

from dataset import load_data
from matrix_factorization_coat import MF_KBIPS_Gau,MF_AKBIPS_Gau,MF_WKBIPS_Gau,MF_KBDR_Gau,MF_AKBDR_Gau,MF_WKBDR_Gau
from matrix_factorization_coat import MF_KBIPS_Exp,MF_AKBIPS_Exp,MF_WKBIPS_Exp,MF_KBDR_Exp,MF_AKBDR_Exp,MF_WKBDR_Exp

from utils import gini_index, ndcg_func, get_user_wise_ctr, rating_mat_to_sample, binarize, shuffle, minU,recall_func, precision_func
mse_func = lambda x,y: np.mean((x-y)**2)
acc_func = lambda x,y: np.sum(x == y) / len(x)

dataset_name = "coat"

In [ ]:
train_mat, test_mat = load_data("coat")        
x_train, y_train = rating_mat_to_sample(train_mat)
x_test, y_test = rating_mat_to_sample(test_mat)
num_user = train_mat.shape[0]
num_item = train_mat.shape[1]

print("# user: {}, # item: {}".format(num_user, num_item))
# binarize
y_train = binarize(y_train)
y_test = binarize(y_test)

### KBIPS

In [ ]:
mf_kbips_gau = MF_KBIPS_Gau(num_user, num_item)
mf_kbips_gau.cuda()

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

mf_kbips_gau.fit(x_train, y_train,  y_ips,
    lr1=3e-2,
    lamb1=1e-3,
    lr2=1e-2,
    lamb2=1e-4,
    gamma = 1,
    batch_size=128,
    J = 3,
    C = 1e-2,
    tol=1e-5)
test_pred = mf_kbips_gau.predict(x_test)
mse_mfkbips = mse_func(y_test, test_pred)
auc_mfkbips = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_kbips_gau, x_test, y_test)
recall_res = recall_func(mf_kbips_gau, x_test, y_test)
precision_res = precision_func(mf_kbips_gau, x_test, y_test)

print("***"*5 + "[MF-KBIPS-Gau]" + "***"*5)
print("[MF-KBIPS-Gau] test mse:", mse_mfkbips)
print("[MF-KBIPS-Gau] test auc:", auc_mfkbips)
print("[MF-KBIPS-Gau] ndcg@5:{:.6f}, ndcg@10:{:.6f}".format(
        np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res["ndcg_10"])))
print("[MF-KBIPS-Gau] recall@5:{:.6f}, recall@10:{:.6f}".format(
        np.mean(recall_res["recall_5"]), np.mean(recall_res["recall_10"])))
print("[MF-KBIPS-Gau] precision@5:{:.6f}, precision@10:{:.6f}".format(
        np.mean(precision_res["precision_5"]), np.mean(precision_res["precision_10"])))
print("[MF-KBIPS-Gau] f1@5:{:.6f}, f1@10:{:.6f}".format(
        2 * (np.mean(precision_res["precision_5"]) * np.mean(recall_res["recall_5"])) / (np.mean(precision_res["precision_5"]) + np.mean(recall_res["recall_5"])), 
        2 * (np.mean(precision_res["precision_10"]) * np.mean(recall_res["recall_10"])) / (np.mean(precision_res["precision_10"]) + np.mean(recall_res["recall_10"]))))

print("***"*5 + "[MF-KBIPS-Gau]" + "***"*5)

In [ ]:
mf_kbips_exp = MF_KBIPS_Exp(num_user, num_item)
mf_kbips_exp.cuda()

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

mf_kbips_exp.fit(x_train, y_train,  y_ips,
    lr1=3e-2,
    lamb1=1e-3,
    lr2=1e-2,
    lamb2=1e-4,
    gamma = 1,
    batch_size=128,
    J = 3,
    C = 1e-2,
    tol=1e-5)
test_pred = mf_kbips_exp.predict(x_test)
mse_mfkbips = mse_func(y_test, test_pred)
auc_mfkbips = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_kbips_exp, x_test, y_test)
recall_res = recall_func(mf_kbips_exp, x_test, y_test)
precision_res = precision_func(mf_kbips_exp, x_test, y_test)

print("***"*5 + "[MF-KBIPS-Exp]" + "***"*5)
print("[MF-KBIPS-Exp] test mse:", mse_mfkbips)
print("[MF-KBIPS-Exp] test auc:", auc_mfkbips)
print("[MF-KBIPS-Exp] ndcg@5:{:.6f}, ndcg@10:{:.6f}".format(
        np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res["ndcg_10"])))
print("[MF-KBIPS-Exp] recall@5:{:.6f}, recall@10:{:.6f}".format(
        np.mean(recall_res["recall_5"]), np.mean(recall_res["recall_10"])))
print("[MF-KBIPS-Exp] precision@5:{:.6f}, precision@10:{:.6f}".format(
        np.mean(precision_res["precision_5"]), np.mean(precision_res["precision_10"])))
print("[MF-KBIPS-Exp] f1@5:{:.6f}, f1@10:{:.6f}".format(
        2 * (np.mean(precision_res["precision_5"]) * np.mean(recall_res["recall_5"])) / (np.mean(precision_res["precision_5"]) + np.mean(recall_res["recall_5"])), 
        2 * (np.mean(precision_res["precision_10"]) * np.mean(recall_res["recall_10"])) / (np.mean(precision_res["precision_10"]) + np.mean(recall_res["recall_10"]))))
print("***"*5 + "[MF-KBIPS-Exp]" + "***"*5)

### AKBIPS

In [ ]:
mf_akbips_Gau = MF_AKBIPS_Gau(num_user, num_item,embedding_k = 4)
mf_akbips_Gau.cuda()

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

mf_akbips_Gau.fit(x_train, y_train,  y_ips,
    lr1=3e-2,
    lamb1=3e-3,
    lr2=3e-2,
    lamb2=1e-3,
    lr3=3e-2,
    lamb3=1e-3,
    gamma = 10,
    batch_size=128,
    num_w_epo = 3,
    J = 3,
    C = 1e-3,
    tol=1e-5)
test_pred = mf_akbips_Gau.predict(x_test)
mse_mfakbips = mse_func(y_test, test_pred)
auc_mfakbips = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_akbips_Gau, x_test, y_test)
recall_res = recall_func(mf_akbips_Gau, x_test, y_test)
precision_res = precision_func(mf_akbips_Gau, x_test, y_test)

print("***"*5 + "[MF-AKBIPS-Gau]" + "***"*5)
print("[MF-AKBIPS-Gau] test mse:", mse_mfakbips)
print("[MF-AKBIPS-Gau] test auc:", auc_mfakbips)
print("[MF-AKBIPS-Gau] ndcg@5:{:.6f}, ndcg@10:{:.6f}".format(
        np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res["ndcg_10"])))
print("[MF-AKBIPS-Gau] recall@5:{:.6f}, recall@10:{:.6f}".format(
        np.mean(recall_res["recall_5"]), np.mean(recall_res["recall_10"])))
print("[MF-AKBIPS-Gau] precision@5:{:.6f}, precision@10:{:.6f}".format(
        np.mean(precision_res["precision_5"]), np.mean(precision_res["precision_10"])))
print("[MF-AKBIPS-Gau] f1@5:{:.6f}, f1@10:{:.6f}".format(
        2 * (np.mean(precision_res["precision_5"]) * np.mean(recall_res["recall_5"])) / (np.mean(precision_res["precision_5"]) + np.mean(recall_res["recall_5"])), 
        2 * (np.mean(precision_res["precision_10"]) * np.mean(recall_res["recall_10"])) / (np.mean(precision_res["precision_10"]) + np.mean(recall_res["recall_10"]))))
print("***"*5 + "[MF-AKBIPS-Gau]" + "***"*5)

In [ ]:
mf_akbips_exp = MF_AKBIPS_Exp(num_user, num_item,embedding_k = 4)
mf_akbips_exp.cuda()

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

mf_akbips_exp.fit(x_train, y_train,  y_ips,
    lr1=3e-2,
    lamb1=3e-3,
    lr2=3e-2,
    lamb2=1e-3,
    lr3=3e-2,
    lamb3=1e-3,
    gamma = 100,
    batch_size=128,
    num_w_epo = 3,
    J = 3,
    C = 1e-3,
    tol=1e-5)
test_pred = mf_akbips_exp.predict(x_test)
mse_mfakbips = mse_func(y_test, test_pred)
auc_mfakbips = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_akbips_exp, x_test, y_test)
recall_res = recall_func(mf_akbips_exp, x_test, y_test)
precision_res = precision_func(mf_akbips_exp, x_test, y_test)

print("***"*5 + "[MF-AKBIPS-Exp]" + "***"*5)
print("[MF-AKBIPS-Exp] test mse:", mse_mfakbips)
print("[MF-AKBIPS-Exp] test auc:", auc_mfakbips)
print("[MF-AKBIPS-Exp] ndcg@5:{:.6f}, ndcg@10:{:.6f}".format(
        np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res["ndcg_10"])))
print("[MF-AKBIPS-Exp] recall@5:{:.6f}, recall@10:{:.6f}".format(
        np.mean(recall_res["recall_5"]), np.mean(recall_res["recall_10"])))
print("[MF-AKBIPS-Exp] precision@5:{:.6f}, precision@10:{:.6f}".format(
        np.mean(precision_res["precision_5"]), np.mean(precision_res["precision_10"])))
print("[MF-AKBIPS-Exp] f1@5:{:.6f}, f1@10:{:.6f}".format(
        2 * (np.mean(precision_res["precision_5"]) * np.mean(recall_res["recall_5"])) / (np.mean(precision_res["precision_5"]) + np.mean(recall_res["recall_5"])), 
        2 * (np.mean(precision_res["precision_10"]) * np.mean(recall_res["recall_10"])) / (np.mean(precision_res["precision_10"]) + np.mean(recall_res["recall_10"]))))
print("***"*5 + "[MF-AKBIPS-Exp]" + "***"*5)

### WKBIPS

In [ ]:
mf_wkbips_gau = MF_WKBIPS_Gau(num_user, num_item)
mf_wkbips_gau.cuda()
ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]
mf_wkbips_gau.fit(x_train, y_train,  y_ips,
    lr1=3e-2,
    lamb1=3e-3,
    lr2=3e-2,
    lamb2=1e-4,
    lr3=3e-2,
    lamb3=1e-3,
    gamma = 1,
    batch_size=128,
    num_w_epo = 3,
    J = 3,
    C = 1e-3,
    tol=1e-5)
test_pred = mf_wkbips_gau.predict(x_test)
mse_mfwkbips = mse_func(y_test, test_pred)
auc_mfwkbips = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_wkbips_gau, x_test, y_test)
recall_res = recall_func(mf_wkbips_gau, x_test, y_test)
precision_res = precision_func(mf_wkbips_gau, x_test, y_test)

print("***"*5 + "[MF-WKBIPS-Gau]" + "***"*5)
print("[MF-WKBIPS-Gau] test mse:", mse_mfwkbips)
print("[MF-WKBIPS-Gau] test auc:", auc_mfwkbips)
print("[MF-WKBIPS-Gau] ndcg@5:{:.6f}, ndcg@10:{:.6f}".format(
        np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res["ndcg_10"])))
print("[MF-WKBIPS-Gau] recall@5:{:.6f}, recall@10:{:.6f}".format(
        np.mean(recall_res["recall_5"]), np.mean(recall_res["recall_10"])))
print("[MF-WKBIPS-Gau] precision@5:{:.6f}, precision@10:{:.6f}".format(
        np.mean(precision_res["precision_5"]), np.mean(precision_res["precision_10"])))
print("[MF-WKBIPS-Gau] f1@5:{:.6f}, f1@10:{:.6f}".format(
        2 * (np.mean(precision_res["precision_5"]) * np.mean(recall_res["recall_5"])) / (np.mean(precision_res["precision_5"]) + np.mean(recall_res["recall_5"])), 
        2 * (np.mean(precision_res["precision_10"]) * np.mean(recall_res["recall_10"])) / (np.mean(precision_res["precision_10"]) + np.mean(recall_res["recall_10"]))))
print("***"*5 + "[MF-WKBIPS-Gau]" + "***"*5)

In [ ]:
mf_wkbips_exp = MF_WKBIPS_Exp(num_user, num_item)
mf_wkbips_exp.cuda()
ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]
mf_wkbips_exp.fit(x_train, y_train,  y_ips,
    lr1=3e-2,
    lamb1=2e-3,
    lr2=3e-2,
    lamb2=1e-4,
    lr3=3e-2,
    lamb3=1e-3,
    gamma = 1,
    batch_size=128,
    num_w_epo = 3,
    J = 3,
    C = 1e-3,
    tol=1e-5)
test_pred = mf_wkbips_exp.predict(x_test)
mse_mfwkbips = mse_func(y_test, test_pred)
auc_mfwkbips = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_wkbips_exp, x_test, y_test)
recall_res = recall_func(mf_wkbips_exp, x_test, y_test)
precision_res = precision_func(mf_wkbips_exp, x_test, y_test)

print("***"*5 + "[MF-WKBIPS-Exp]" + "***"*5)
print("[MF-WKBIPS-Exp] test mse:", mse_mfwkbips)
print("[MF-WKBIPS-Exp] test auc:", auc_mfwkbips)
print("[MF-WKBIPS-Exp] ndcg@5:{:.6f}, ndcg@10:{:.6f}".format(
        np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res["ndcg_10"])))
print("[MF-WKBIPS-Exp] recall@5:{:.6f}, recall@10:{:.6f}".format(
        np.mean(recall_res["recall_5"]), np.mean(recall_res["recall_10"])))
print("[MF-WKBIPS-Exp] precision@5:{:.6f}, precision@10:{:.6f}".format(
        np.mean(precision_res["precision_5"]), np.mean(precision_res["precision_10"])))
print("[MF-WKBIPS-Exp] f1@5:{:.6f}, f1@10:{:.6f}".format(
        2 * (np.mean(precision_res["precision_5"]) * np.mean(recall_res["recall_5"])) / (np.mean(precision_res["precision_5"]) + np.mean(recall_res["recall_5"])), 
        2 * (np.mean(precision_res["precision_10"]) * np.mean(recall_res["recall_10"])) / (np.mean(precision_res["precision_10"]) + np.mean(recall_res["recall_10"]))))
print("***"*5 + "[MF-WKBIPS-Exp]" + "***"*5)

### KBDR

In [ ]:
mf_kbdr_gau = MF_KBDR_Gau(num_user, num_item)
mf_kbdr_gau.cuda()

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

mf_kbdr_gau.fit(x_train, y_train, y_ips, 
    lr=1e-2,
    lamb=3e-3,
    lr2=1e-2,
    lamb2=1e-5,
    gamma=1,
    batch_size=128,
    J=3,
    C=1e-4,
    tol=1e-5)
test_pred = mf_kbdr_gau.predict(x_test)
mse_mfkbdr = mse_func(y_test, test_pred)
auc_mfkbdr = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_kbdr_gau, x_test, y_test)
recall_res = recall_func(mf_kbdr_gau, x_test, y_test)
precision_res = precision_func(mf_kbdr_gau, x_test, y_test)

print("***"*5 + "[MF-KBDR-Gau]" + "***"*5)
print("[MF-KBDR-Gau] test mse:", mse_mfkbdr)
print("[MF-KBDR-Gau] test auc:", auc_mfkbdr)
print("[MF-KBDR-Gau] ndcg@5:{:.6f}, ndcg@10:{:.6f}".format(
        np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res["ndcg_10"])))
print("[MF-KBDR-Gau] recall@5:{:.6f}, recall@10:{:.6f}".format(
        np.mean(recall_res["recall_5"]), np.mean(recall_res["recall_10"])))
print("[MF-KBDR-Gau] precision@5:{:.6f}, precision@10:{:.6f}".format(
        np.mean(precision_res["precision_5"]), np.mean(precision_res["precision_10"])))
print("[MF-KBDR-Gau] f1@5:{:.6f}, f1@10:{:.6f}".format(
        2 * (np.mean(precision_res["precision_5"]) * np.mean(recall_res["recall_5"])) / (np.mean(precision_res["precision_5"]) + np.mean(recall_res["recall_5"])), 
        2 * (np.mean(precision_res["precision_10"]) * np.mean(recall_res["recall_10"])) / (np.mean(precision_res["precision_10"]) + np.mean(recall_res["recall_10"]))))
print("***"*5 + "[MF-KBDR-Gau]" + "***"*5)

In [ ]:
mf_kbdr_exp = MF_KBDR_Exp(num_user, num_item)
mf_kbdr_exp.cuda()

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

mf_kbdr_exp.fit(x_train, y_train, y_ips, 
    lr=1e-2,
    lamb=3e-3,
    lr2=1e-2,
    lamb2=1e-5,
    gamma=1,
    batch_size=128,
    J=3,
    C=1e-4,
    tol=1e-5)
test_pred = mf_kbdr_exp.predict(x_test)
mse_mfkbdr = mse_func(y_test, test_pred)
auc_mfkbdr = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_kbdr_exp, x_test, y_test)
recall_res = recall_func(mf_kbdr_exp, x_test, y_test)
precision_res = precision_func(mf_kbdr_exp, x_test, y_test)

print("***"*5 + "[MF-KBDR-Exp]" + "***"*5)
print("[MF-KBDR-Exp] test mse:", mse_mfkbdr)
print("[MF-KBDR-Exp] test auc:", auc_mfkbdr)
print("[MF-KBDR-Exp] ndcg@5:{:.6f}, ndcg@10:{:.6f}".format(
        np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res["ndcg_10"])))
print("[MF-KBDR-Exp] recall@5:{:.6f}, recall@10:{:.6f}".format(
        np.mean(recall_res["recall_5"]), np.mean(recall_res["recall_10"])))
print("[MF-KBDR-Exp] precision@5:{:.6f}, precision@10:{:.6f}".format(
        np.mean(precision_res["precision_5"]), np.mean(precision_res["precision_10"])))
print("[MF-KBDR-Exp] f1@5:{:.6f}, f1@10:{:.6f}".format(
        2 * (np.mean(precision_res["precision_5"]) * np.mean(recall_res["recall_5"])) / (np.mean(precision_res["precision_5"]) + np.mean(recall_res["recall_5"])), 
        2 * (np.mean(precision_res["precision_10"]) * np.mean(recall_res["recall_10"])) / (np.mean(precision_res["precision_10"]) + np.mean(recall_res["recall_10"]))))
print("***"*5 + "[MF-KBDR-Exp]" + "***"*5)

### AKBDR

In [ ]:
mf_akbdr_gau = MF_AKBDR_Gau(num_user, num_item)
mf_akbdr_gau.cuda()

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

mf_akbdr_gau.fit(x_train, y_train, y_ips, 
    lr=3e-2,
    lamb=5e-3,
    lr1=3e-2,
    lamb1=3e-3,
    lr2=3e-2,
    lamb2=1e-3,
    lr3=3e-2,
    lamb3=1e-3,
    gamma=10,
    batch_size=128,
    num_w_epo = 3,
    J = 3,
    C = 1e-3,
    tol=1e-5)


test_pred = mf_akbdr_gau.predict(x_test)
mse_mfakbdr = mse_func(y_test, test_pred)
auc_mfakbdr = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_akbdr_gau, x_test, y_test)
recall_res = recall_func(mf_akbdr_gau, x_test, y_test)
precision_res = precision_func(mf_akbdr_gau, x_test, y_test)

print("***"*5 + "[MF-AKBDR-Gau]" + "***"*5)
print("[MF-AKBDR-Gau] test mse:", mse_mfakbdr)
print("[MF-AKBDR-Gau] test auc:", auc_mfakbdr)
print("[MF-AKBDR-Gau] ndcg@5:{:.6f}, ndcg@10:{:.6f}".format(
        np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res["ndcg_10"])))
print("[MF-AKBDR-Gau] recall@5:{:.6f}, recall@10:{:.6f}".format(
        np.mean(recall_res["recall_5"]), np.mean(recall_res["recall_10"])))
print("[MF-AKBDR-Gau] precision@5:{:.6f}, precision@10:{:.6f}".format(
        np.mean(precision_res["precision_5"]), np.mean(precision_res["precision_10"])))
print("[MF-AKBDR-Gau] f1@5:{:.6f}, f1@10:{:.6f}".format(
        2 * (np.mean(precision_res["precision_5"]) * np.mean(recall_res["recall_5"])) / (np.mean(precision_res["precision_5"]) + np.mean(recall_res["recall_5"])), 
        2 * (np.mean(precision_res["precision_10"]) * np.mean(recall_res["recall_10"])) / (np.mean(precision_res["precision_10"]) + np.mean(recall_res["recall_10"]))))
print("***"*5 + "[MF-AKBDR-Gau]" + "***"*5)

In [ ]:
mf_akbdr_exp = MF_AKBDR_Exp(num_user, num_item)
mf_akbdr_exp.cuda()

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

mf_akbdr_exp.fit(x_train, y_train, y_ips, 
    lr=3e-2,
    lamb=5e-3,
    lr1=3e-2,
    lamb1=3e-3,
    lr2=3e-2,
    lamb2=1e-3,
    lr3=3e-2,
    lamb3=1e-3,
    gamma=10,
    batch_size=128,
    num_w_epo = 3,
    J = 3,
    C = 1e-3,
    tol=1e-5)
test_pred = mf_akbdr_exp.predict(x_test)
mse_mfakbdr = mse_func(y_test, test_pred)
auc_mfakbdr = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_akbdr_exp, x_test, y_test)
recall_res = recall_func(mf_akbdr_exp, x_test, y_test)
precision_res = precision_func(mf_akbdr_exp, x_test, y_test)

print("***"*5 + "[MF-AKBDR-Exp]" + "***"*5)
print("[MF-AKBDR-Exp] test mse:", mse_mfakbdr)
print("[MF-AKBDR-Exp] test auc:", auc_mfakbdr)
print("[MF-AKBDR-Exp] ndcg@5:{:.6f}, ndcg@10:{:.6f}".format(
        np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res["ndcg_10"])))
print("[MF-AKBDR-Exp] recall@5:{:.6f}, recall@10:{:.6f}".format(
        np.mean(recall_res["recall_5"]), np.mean(recall_res["recall_10"])))
print("[MF-AKBDR-Exp] precision@5:{:.6f}, precision@10:{:.6f}".format(
        np.mean(precision_res["precision_5"]), np.mean(precision_res["precision_10"])))
print("[MF-AKBDR-Exp] f1@5:{:.6f}, f1@10:{:.6f}".format(
        2 * (np.mean(precision_res["precision_5"]) * np.mean(recall_res["recall_5"])) / (np.mean(precision_res["precision_5"]) + np.mean(recall_res["recall_5"])), 
        2 * (np.mean(precision_res["precision_10"]) * np.mean(recall_res["recall_10"])) / (np.mean(precision_res["precision_10"]) + np.mean(recall_res["recall_10"]))))
print("***"*5 + "[MF-AKBDR-Exp]" + "***"*5)

### WKBDR

In [ ]:
mf_wkbdr_gau = MF_WKBDR_Gau(num_user, num_item)
mf_wkbdr_gau.cuda()

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]
mf_wkbdr_gau.fit(x_train, y_train,  y_ips,
    lr=3e-2,
    lamb=3e-3,
    lr2=1e-2,
    lamb2=1e-5,
    gamma=1,
    batch_size=128,
    J = 3,
    C = 1e-4,
    tol=1e-5)
test_pred = mf_wkbdr_gau.predict(x_test)
mse_mfwkbdr = mse_func(y_test, test_pred)
auc_mfwkbdr = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_wkbdr_gau, x_test, y_test)
recall_res = recall_func(mf_wkbdr_gau, x_test, y_test)
precision_res = precision_func(mf_wkbdr_gau, x_test, y_test)

print("***"*5 + "[MF-WKBDR-Gau]" + "***"*5)
print("[MF-WKBDR-Gau] test mse:", mse_mfwkbdr)
print("[MF-WKBDR-Gau] test auc:", auc_mfwkbdr)
print("[MF-WKBDR-Gau] ndcg@5:{:.6f}, ndcg@10:{:.6f}".format(
        np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res["ndcg_10"])))
print("[MF-WKBDR-Gau] recall@5:{:.6f}, recall@10:{:.6f}".format(
        np.mean(recall_res["recall_5"]), np.mean(recall_res["recall_10"])))
print("[MF-WKBDR-Gau] precision@5:{:.6f}, precision@10:{:.6f}".format(
        np.mean(precision_res["precision_5"]), np.mean(precision_res["precision_10"])))
print("[MF-WKBDR-Gau] f1@5:{:.6f}, f1@10:{:.6f}".format(
        2 * (np.mean(precision_res["precision_5"]) * np.mean(recall_res["recall_5"])) / (np.mean(precision_res["precision_5"]) + np.mean(recall_res["recall_5"])), 
        2 * (np.mean(precision_res["precision_10"]) * np.mean(recall_res["recall_10"])) / (np.mean(precision_res["precision_10"]) + np.mean(recall_res["recall_10"]))))
print("***"*5 + "[MF-WKBDR-Gau]" + "***"*5)

In [ ]:
mf_wkbdr_exp = MF_WKBDR_Exp(num_user, num_item)
mf_wkbdr_exp.cuda()

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]
mf_wkbdr_exp.fit(x_train, y_train,  y_ips,
    lr=3e-2,
    lamb=3e-3,
    lr2=1e-2,
    lamb2=1e-5,
    gamma=1,
    batch_size=128,
    J = 3,
    C = 1e-4,
    tol=1e-5)
test_pred = mf_wkbdr_exp.predict(x_test)
mse_mfwkbdr = mse_func(y_test, test_pred)
auc_mfwkbdr = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_wkbdr_exp, x_test, y_test)
recall_res = recall_func(mf_wkbdr_exp, x_test, y_test)
precision_res = precision_func(mf_wkbdr_exp, x_test, y_test)

print("***"*5 + "[MF-WKBDR-Exp]" + "***"*5)
print("[MF-WKBDR-Exp] test mse:", mse_mfwkbdr)
print("[MF-WKBDR-Exp] test auc:", auc_mfwkbdr)
print("[MF-WKBDR-Exp] ndcg@5:{:.6f}, ndcg@10:{:.6f}".format(
        np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res["ndcg_10"])))
print("[MF-WKBDR-Exp] recall@5:{:.6f}, recall@10:{:.6f}".format(
        np.mean(recall_res["recall_5"]), np.mean(recall_res["recall_10"])))
print("[MF-WKBDR-Exp] precision@5:{:.6f}, precision@10:{:.6f}".format(
        np.mean(precision_res["precision_5"]), np.mean(precision_res["precision_10"])))
print("[MF-WKBDR-Exp] f1@5:{:.6f}, f1@10:{:.6f}".format(
        2 * (np.mean(precision_res["precision_5"]) * np.mean(recall_res["recall_5"])) / (np.mean(precision_res["precision_5"]) + np.mean(recall_res["recall_5"])), 
        2 * (np.mean(precision_res["precision_10"]) * np.mean(recall_res["recall_10"])) / (np.mean(precision_res["precision_10"]) + np.mean(recall_res["recall_10"]))))
print("***"*5 + "[MF-WKBDR-Exp]" + "***"*5)